In [ ]:
!rm -rf Fashion-U-Want-Virtual-Try-On
!git clone https://github.com/PARKYUNSU/Fashion-U-Want-Virtual-Try-On

!pip install numpy==1.24.4 scipy==1.13.3 --no-cache-dir --force-reinstall

!pip install carvekit

!pip install tensorboardX av torchgeometry iglovikov_helper_functions albumentations

%cd Fashion-U-Want-Virtual-Try-On

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import os
import gdown
from concurrent.futures import ThreadPoolExecutor

# 필수 의존성 패키지들을 하나의 명령어로 설치
!apt-get update && apt-get install -y \
    cmake \
    libatlas-base-dev \
    gfortran \
    libopencv-dev \
    libopenblas-dev \
    liblapack-dev \
    build-essential \
    libprotobuf-dev \
    protobuf-compiler \
    libgoogle-glog-dev \
    libgflags-dev \
    libhdf5-dev \
    libleveldb-dev \
    liblmdb-dev \
    libsnappy-dev \
    libboost-all-dev

# OpenPose clone
!rm -rf openpose
!git clone --depth 1 https://github.com/AlecDusheck/openpose.git
%cd openpose

# 모델 디렉토리 한 번에 생성
model_dirs = [
    'models/pose/body_25',
    'models/face',
    'models/hand',
    'models/pose/coco',
    'models/pose/mpi'
]
for dir_path in model_dirs:
    os.makedirs(dir_path, exist_ok=True)

# 모델 파일 정보
model_files = {
    "1cf-eHJhd2_GXLY33pSVH1Bu4aCTYbhmk": "models/hand/pose_iter_102000.caffemodel",
    "1kC6_AFJtEcvjE_OwJ-OtWmUPq9RUoNy1": "models/face/pose_iter_116000.caffemodel",
    "1HLFUOewfg9f50dlvxGwjNIDcPb9EfzGd": "models/pose/mpi/pose_iter_160000.caffemodel",
    "1tdfElEu2Quc0dBavn4RCal899r9XeEiS": "models/pose/coco/pose_iter_440000.caffemodel",
    "17SOFB6MXFHViYmhzAV0LguoRYJrJMF4e": "models/pose/body_25/pose_iter_584000.caffemodel"
}

# 병렬로 모델 다운로드
def download_model(file_info):
    file_id, output_path = file_info
    print(f"Downloading {output_path}...")
    gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=True)

with ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(download_model, model_files.items())

# CMakeLists.txt 최적화
cmake_path = "openpose/CMakeLists.txt"
if os.path.exists(cmake_path):
    with open(cmake_path, "r") as file:
        cmake_content = file.read()
    
    # 한 번에 파일 수정
    cmake_content = '\n'.join([line for line in cmake_content.split('\n') 
                              if "download_model" not in line])
    
    with open(cmake_path, "w") as file:
        file.write(cmake_content)
    print("Modified CMakeLists.txt to skip model downloads.")

# OpenPose 빌드 최적화
!mkdir -p build
%cd build
!cmake -DBUILD_PYTHON=ON \
       -DCMAKE_BUILD_TYPE=Release \
       -DBUILD_EXAMPLES=OFF \
       -DBUILD_DOCS=OFF \
       -DBUILD_TUTORIALS=OFF ..

# 사용 가능한 CPU 코어 수를 확인하고 빌드에 활용
import multiprocessing
cpu_count = multiprocessing.cpu_count()
!make -j{cpu_count}

os.chdir("../../")

In [ ]:
# import subprocess
# import os
# import gdown

# # 로그를 숨기는 함수
# def run_command(command):
#     result = subprocess.run(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
#     if result.returncode != 0:
#         print(f"Command failed: {command}")

# # 의존성 설치
# run_command("apt-get update")
# run_command("apt-get install -y cmake libatlas-base-dev gfortran libopencv-dev libopenblas-dev liblapack-dev")
# run_command("apt-get install -y build-essential libprotobuf-dev protobuf-compiler")
# run_command("apt-get install -y libgoogle-glog-dev libgflags-dev libhdf5-dev libleveldb-dev liblmdb-dev libsnappy-dev")
# run_command("apt-get install -y libboost-all-dev")

# # OpenPose 클론
# run_command("rm -rf openpose")
# run_command("git clone https://github.com/AlecDusheck/openpose.git")

# os.chdir("openpose")

# # OpenPose 모델 다운로드
# os.makedirs('models/pose/body_25', exist_ok=True)
# gdown.download("https://drive.google.com/uc?id=17SOFB6MXFHViYmhzAV0LguoRYJrJMF4e", "models/pose/body_25/pose_iter_584000.caffemodel", quiet=True)

# # CMakeLists.txt 수정
# cmake_file = "CMakeLists.txt"
# if os.path.exists(cmake_file):
#     with open(cmake_file, "r") as file:
#         cmake_content = file.readlines()
#     with open(cmake_file, "w") as file:
#         for line in cmake_content:
#             if "download_model" not in line:
#                 file.write(line)

# # OpenPose 빌드
# os.makedirs("build", exist_ok=True)
# os.chdir("build")
# run_command("cmake ..")
# run_command("make -j$(nproc)")
# os.chdir("../../")

In [ ]:
!pip install --upgrade --no-cache-dir gdown

%cd HR-VITON
!gdown https://drive.google.com/u/0/uc?id=1T5_YDUhYSSKPC_nZMk2NeC-XXUFoYeNy&export=download
!gdown https://drive.google.com/u/0/uc?id=1XJTCdRBOPVgVTmqzhVGFAgMm2NLkw5uQ&export=download
%cd ../

%cd Graphonomy-master
!gdown https://drive.google.com/u/0/uc?id=1eUe18HoH05p0yFUd_sN6GXdTj82aW0m9&export=download
%cd ../

# Cloth
Upload the image of the clothing you want to apply!

In [ ]:
import os
import shutil
from google.colab import files

input_dir = 'input'
uploaded = files.upload()
for filename in uploaded.keys():
  input_path = os.path.join(input_dir, filename)
  shutil.move(filename, input_path)
os.rename(input_path, input_dir+'/cloth.jpg')

# Person
Upload the image of the person or model you'd like to use!

In [ ]:
input_dir = 'input'
uploaded = files.upload()
for filename in uploaded.keys():
  input_path = os.path.join(input_dir, filename)
  shutil.move(filename, input_path)
os.rename(input_path, input_dir+'/model.jpg')

In [ ]:
import matplotlib.pyplot as plt
import cv2

mode = cv2.cvtColor(cv2.imread("./input/model.jpg"), cv2.COLOR_BGR2RGB)
cloth = cv2.cvtColor(cv2.imread("./input/cloth.jpg"), cv2.COLOR_BGR2RGB)
## Display Images
fig, axes = plt.subplots(nrows=1, ncols=2)
dpi = fig.get_dpi()
fig.set_size_inches(900/ dpi, 448 / dpi)
plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
axes[0].axis('off')
axes[0].imshow(mode)
axes[1].axis('off')
axes[1].imshow(cloth)
plt.show()

In [ ]:
pwd

In [ ]:
!python main.py #--background False

In [ ]:
from PIL import Image
from IPython.display import Image

image1 = Image(filename="./input/finalimg.png")
display(image1)